In [2]:
import pandas as pd, tweepy
import nump

consumer_key = 'B3Xa8vhhGkN6VFxrn4nKkxws8';
consumer_secret ='Y5RytaByPpJkk3aGjTfHFpR1JrcU7JC1TGuprbjwSMXAMA7sBT';
access_token = '1314362391837265922-X11kfZOaBkIfAJEy1NLGB10cZfeW0b';
access_token_secret = '20PWHZ3uKq3g0haZFTzFiqty1QTnTZtl76smZkinGt7HA';

auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

def fetch_tw(ids, tweet_excel):
    list_of_tw_status = api.statuses_lookup(ids, tweet_mode= "extended")
    empty_data = pd.DataFrame()
    for status in list_of_tw_status:
            tweet_elem = {"tweet_id": status.id,
                     "screen_name": status.user.screen_name,
                     "tweet":status.full_text,
                     "date":status.created_at,
                     "favorites":status.favorite_count,
                     "retweets": status.retweet_count,
                     "retweeted": status.retweeted,
                     "user_followers": status.user.followers_count,
                     "user_verified": status.user.verified,
                          
                         }
            empty_data = empty_data.append(tweet_elem, ignore_index = True)
    empty_data.to_csv(tweet_excel, mode="a")
    

def tweet_text(tweet_links, tweet_csv):
    tweet_url = pd.read_csv(tweet_links, index_col= None, header = None, names = ["links"])
    tweet_url.head()
    af = lambda x: x["links"].split("/")[-1]
    tweet_url['id'] = tweet_url.apply(af, axis=1)
    ids = tweet_url['id'].tolist()
    total_count = len(ids)
    chunks = (total_count - 1) // 50 + 1

    for i in range(chunks):
            batch = ids[i*50:(i+1)*50]
            result = fetch_tw(batch, tweet_csv)    
            
tweet_text("Tesla_tweets.txt","Tesla_tweets.csv");

In [ ]:
tweet_text("volkswagen_tweetsmax_2015.txt","volkswagen_tweets.csv");

In [7]:
tweet_text("wellsfargo_tweets_2016.txt","wellsfargo_tweets.csv");

In [ ]:
tweet_text("equifax_tweets_2017.txt","equifax_tweets.csv");

In [29]:
import numpy as np
import re
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt
def clean_tweets(lst):
    # remove twitter Return handles (RT @xxx:)
    lst = np.vectorize(remove_pattern)(lst, "RT @[\w]*:")
    # remove twitter handles (@xxx)
    lst = np.vectorize(remove_pattern)(lst, "@[\w]*")
    # remove URL links (httpxxx)
    lst = np.vectorize(remove_pattern)(lst, "https?://[A-Za-z0-9./]*")
    # remove special characters, numbers, punctuations (except for #)
    lst = np.core.defchararray.replace(lst, "[^a-zA-Z#]", " ")
    return lst
def sentiment_analyzer_scores(text):
    score = analyser.polarity_scores(text)
    lb = score['compound']
    if lb >= 0.05:
        return 1
    elif (lb > -0.05) and (lb < 0.05):
        return 0
    else:
        return -1

In [3]:
df = pd.read_csv('volkswagen_tweets.csv')
print(df)

      Unnamed: 0                 date favorites retweeted retweets  \
0              0  2015-09-30 23:59:01       0.0       0.0      0.0   
1              1  2015-09-30 23:59:11       0.0       0.0      0.0   
2              2  2015-09-30 23:49:47       0.0       0.0      0.0   
3              3  2015-09-30 23:58:58       0.0       0.0      0.0   
4              4  2015-09-30 23:55:09       0.0       0.0      0.0   
...          ...                  ...       ...       ...      ...   
10087         39  2015-09-30 05:01:37       0.0       0.0      0.0   
10088         40  2015-09-30 04:52:01       0.0       0.0      0.0   
10089         41  2015-09-30 04:54:35       0.0       0.0      0.0   
10090         42  2015-09-30 04:48:08       0.0       0.0      0.0   
10091         43  2015-09-30 04:54:07       0.0       0.0      0.0   

          screen_name                                              tweet  \
0       tuitAustralia  Volkswagen scandal: Choice car test points to ...   
1      

In [22]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [48]:
import csv
count = 0 
tweet_label = 6
favorites = 2
retweeted = 3
retweets = 4
user_followers = 8
user_verified = 9

with open('volkswagen_tweets.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        if(count < 5):
            count = count + 1
            print('Raw Tweet:', row[tweet_label])
            ## need to unvectorize to use
            print('Cleaned Tweet:', clean_tweets(row[tweet_label]))
            polarity = analyser.polarity_scores(row[tweet_label])
            print('Favorites:', row[favorites], ' Retweeted:', row[retweeted], ' Retweets:', row[retweets], ' User Followers:', row[user_followers], 
                  ' User_Verified:',row[user_verified])
            print(polarity['compound'])
            print('')
            
           

Raw Tweet: tweet
Cleaned Tweet: tweet
Favorites: favorites  Retweeted: retweeted  Retweets: retweets  User Followers: user_followers  User_Verified: user_verified
0.0

Raw Tweet: Understanding The 'Test Mode' That Let VW Trick The Emissions Test 
Volkswagen faces a max… http://t.co/FWbNIlU2VF http://t.co/L68azsUnSz
Cleaned Tweet: Understanding The 'Test Mode' That Let VW Trick The Emissions Test 
Volkswagen faces a max…  
Favorites: 0.0  Retweeted: 0.0  Retweets: 0.0  User Followers: 10481.0  User_Verified: 0.0
-0.0516

Raw Tweet: Volkswagen Was "So Close" To Joining Formula One Says Mosley: Max Mosley, former president of Formula One’s go... http://t.co/AZ090pSI9K
Cleaned Tweet: Volkswagen Was "So Close" To Joining Formula One Says Mosley: Max Mosley, former president of Formula One’s go... 
Favorites: 0.0  Retweeted: 0.0  Retweets: 0.0  User Followers: 1270.0  User_Verified: 0.0
0.0

Raw Tweet: Volkswagen in deep trouble as diesel fuel emission scandal spreads to Europe - Daily Sabah